In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [3]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [4]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test = fill_missing_values(X_test)
print(X_train.shape)
print(X_test.shape)

(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [5]:
X_train,y_train = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1199, 828)


## 3. Feature Selection

In [6]:
# X, X_test = feature_reduction(X, X_test,750)
X_train, X_test = select_features(X_train, y_train, X_test,feature_num = 50)
print("Traing data shape after selection: {}".format(X_train.shape))
print("Testing data shape after selection: {}".format(X_test.shape))

Traing data shape after selection: (1199, 50)
Testing data shape after selection: (776, 50)


## 4. Extra Tree

In [7]:
def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
   
#     params = {
#         'task': 'train',
#         'boosting_type': 'gbdt',
#         'objective': 'regression',
#         # overfitting
#         'lambda_l1': 0.022311098951506406,
#         'lambda_l2': 0.024242524864550277,
#         # randomly select feature to be used for each round
#         'feature_fraction': 0.5005532443822837,
#         # percentage of rows used per tree building iteration
#         'subsample': 0.8587066270194157,
#         # recommended to use smaller learning rate with larger iteration
#         'learning_rate': 0.04199888269541904,
#         # control the model complexity
#         'num_leaves': 1760,
#         'max_depth': 14,

#         'num_iterations':483,
        
#         # Start with a lower number of trees to build a baseline and increase it later
#         'n_estimators': 1250,
        
#         'min_child_weight': 1,
        
#         'colsample_bytree': 0.8,
        
#         'bagging_fraction': 0.8,
#         'bagging_freq': 5,
#         'verbose': -1,
#         # Small max_bin causes faster speed and large value improves accuracy
#         'max_bin':255,
#     }
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
        # overfitting
    'lambda_l1':0.0,
    'lambda_l2':0.0,
    # control the model complexity
    'num_leaves': 1700,
    'max_depth': 11,
    # randomly select feature to be used for each round
    'feature_fraction': 0.5,
    # percentage of rows used per tree building iteration
    'subsample': 0.8,
    # Small max_bin causes faster speed and large value improves accuracy
    'max_bin':255,
    # recommended to use smaller learning rate with larger iteration
    'learning_rate': 0.025,
    'num_iterations':600,
    # Start with a lower number of trees to build a baseline and increase it later
    'n_estimators': 1000,
        
    'min_child_weight': 1,
    'colsample_bytree': 0.8,
        
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    }
    
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=483,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=48,
                    verbose_eval=True
                   )
    
    y_val_pred = gbm.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    y_pred = gbm.predict(X_test) 
    
    return score, y_pred

def train_k_fold(X, y, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_val)

        print('The obtained validation r2 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))

def train_k_fold_predict(X, y, X_test, fold_num=10):
    kf = KFold(n_splits=fold_num)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    cnt = 0
    val_score = 0.0
        
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
        val_score += score
        if(score > 0.6):
            y_test_predict += y_pred
            cnt += 1
    return val_score/fold_num, y_test_predict/cnt

In [8]:
train_k_fold(X_train,y_train, fold_num=10) 

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000859 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 1079, number of used features: 50
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: featur

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[64]	training's l2: 26.5233	training's r2: 0.723276	valid_1's l2: 32.5934	valid_1's r2: 0.616185
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[65]	training's l2: 26.1747	training's r2: 0.726913	valid_1's l2: 32.2552	valid_1's r2: 0.620169
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[66]	training's l2: 25.8294	training's r2: 0.730516	valid_1's l2: 32.1567	valid_1's r2: 0.621329
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[67]	training's l2: 25.4928	training's r2: 0.734028	valid_1's l2: 31.8944	valid_1's r2: 0.624418
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[68]	training's l2: 25.2031	training's r2: 0.73705	valid_1's l2: 31.8228	valid_1's r2: 0.62526
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[69]	training's l2: 24.8943	training's r2: 0.740272	valid_1's l2: 3

[117]	training's l2: 15.3363	training's r2: 0.839993	valid_1's l2: 27.1709	valid_1's r2: 0.68004
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[118]	training's l2: 15.1876	training's r2: 0.841544	valid_1's l2: 27.118	valid_1's r2: 0.680664
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[119]	training's l2: 15.0481	training's r2: 0.843	valid_1's l2: 26.9973	valid_1's r2: 0.682084
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[120]	training's l2: 14.92	training's r2: 0.844337	valid_1's l2: 26.9892	valid_1's r2: 0.682179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[121]	training's l2: 14.8176	training's r2: 0.845405	valid_1's l2: 26.9572	valid_1's r2: 0.682557
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[122]	training's l2: 14.7139	training's r2: 0.846486	valid_1's l2: 26.9042	valid_1's r2: 0.683181
[LightGBM] [Warning] No further splits with 

[197]	training's l2: 9.37849	training's r2: 0.902152	valid_1's l2: 24.8062	valid_1's r2: 0.707887
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[198]	training's l2: 9.34823	training's r2: 0.902468	valid_1's l2: 24.7861	valid_1's r2: 0.708123
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[199]	training's l2: 9.30649	training's r2: 0.902903	valid_1's l2: 24.7888	valid_1's r2: 0.708091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	training's l2: 9.25675	training's r2: 0.903422	valid_1's l2: 24.7901	valid_1's r2: 0.708076
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[201]	training's l2: 9.19375	training's r2: 0.90408	valid_1's l2: 24.7713	valid_1's r2: 0.708298
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[202]	training's l2: 9.13842	training's r2: 0.904657	valid_1's l2: 24.7245	valid_1's r2: 0.708848
[LightGBM] [Warning] No further splits

[252]	training's l2: 7.12356	training's r2: 0.925678	valid_1's l2: 24.2159	valid_1's r2: 0.714838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[253]	training's l2: 7.09807	training's r2: 0.925944	valid_1's l2: 24.2364	valid_1's r2: 0.714597
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[254]	training's l2: 7.07913	training's r2: 0.926142	valid_1's l2: 24.219	valid_1's r2: 0.714802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[255]	training's l2: 7.05334	training's r2: 0.926411	valid_1's l2: 24.2285	valid_1's r2: 0.714689
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[256]	training's l2: 7.03146	training's r2: 0.926639	valid_1's l2: 24.2005	valid_1's r2: 0.715019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[257]	training's l2: 7.0048	training's r2: 0.926917	valid_1's l2: 24.1749	valid_1's r2: 0.715321
[LightGBM] [Warning] No further splits 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[223]	training's l2: 7.09193	training's r2: 0.925157	valid_0's l2: 46.4525	valid_0's r2: 0.51787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[224]	training's l2: 7.06747	training's r2: 0.925415	valid_0's l2: 46.4073	valid_0's r2: 0.518339
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[225]	training's l2: 7.02368	training's r2: 0.925877	valid_0's l2: 46.4522	valid_0's r2: 0.517873
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[226]	training's l2: 6.97788	training's r2: 0.926361	valid_0's l2: 46.3986	valid_0's r2: 0.518429
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[227]	training's l2: 6.93897	training's r2: 0.926772	valid_0's l2: 46.3503	valid_0's r2: 0.51893
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[228]	training's l2: 6.89964	training's r2: 0.927187	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[286]	training's l2: 5.19079	training's r2: 0.94522	valid_0's l2: 45.341	valid_0's r2: 0.529406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[287]	training's l2: 5.16438	training's r2: 0.945499	valid_0's l2: 45.3875	valid_0's r2: 0.528923
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[288]	training's l2: 5.13952	training's r2: 0.945761	valid_0's l2: 45.4478	valid_0's r2: 0.528298
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[289]	training's l2: 5.10577	training's r2: 0.946118	valid_0's l2: 45.4747	valid_0's r2: 0.528018
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[290]	training's l2: 5.07514	training's r2: 0.946441	valid_0's l2: 45.4213	valid_0's r2: 0.528572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[291]	training's l2: 5.04851	training's r2: 0.946722	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[347]	training's l2: 3.99468	training's r2: 0.957843	valid_0's l2: 44.6859	valid_0's r2: 0.536205
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[348]	training's l2: 3.98298	training's r2: 0.957967	valid_0's l2: 44.6563	valid_0's r2: 0.536512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[349]	training's l2: 3.96896	training's r2: 0.958115	valid_0's l2: 44.6215	valid_0's r2: 0.536873
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[350]	training's l2: 3.95598	training's r2: 0.958252	valid_0's l2: 44.6069	valid_0's r2: 0.537025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[351]	training's l2: 3.94245	training's r2: 0.958394	valid_0's l2: 44.6487	valid_0's r2: 0.536591
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[352]	training's l2: 3.92023	training's r2: 0.958629	valid_0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[412]	training's l2: 3.17161	training's r2: 0.966529	valid_0's l2: 44.2325	valid_0's r2: 0.540911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[413]	training's l2: 3.16064	training's r2: 0.966645	valid_0's l2: 44.2484	valid_0's r2: 0.540746
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[414]	training's l2: 3.15286	training's r2: 0.966727	valid_0's l2: 44.2523	valid_0's r2: 0.540705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[415]	training's l2: 3.13922	training's r2: 0.966871	valid_0's l2: 44.2408	valid_0's r2: 0.540825
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[416]	training's l2: 3.12932	training's r2: 0.966976	valid_0's l2: 44.2672	valid_0's r2: 0.540551
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[417]	training's l2: 3.11977	training's r2: 0.967076	valid_0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[489]	training's l2: 2.47303	training's r2: 0.973902	valid_0's l2: 44.134	valid_0's r2: 0.541933
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[490]	training's l2: 2.46513	training's r2: 0.973985	valid_0's l2: 44.1805	valid_0's r2: 0.54145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[491]	training's l2: 2.45737	training's r2: 0.974067	valid_0's l2: 44.175	valid_0's r2: 0.541508
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[492]	training's l2: 2.44759	training's r2: 0.97417	valid_0's l2: 44.1998	valid_0's r2: 0.54125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[493]	training's l2: 2.43845	training's r2: 0.974266	valid_0's l2: 44.1942	valid_0's r2: 0.541308
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[494]	training's l2: 2.42907	training's r2: 0.974365	valid_0's l2

In [9]:
_, y_test_pred = train_k_fold_predict(X_train, y_train, X_test)
final_res = np.vstack((indices_test, y_test_pred)).T

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000967 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 1079, number of used features: 50
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: featur

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[67]	training's l2: 25.4928	training's r2: 0.734028	valid_0's l2: 31.8944	valid_0's r2: 0.624418
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[68]	training's l2: 25.2031	training's r2: 0.73705	valid_0's l2: 31.8228	valid_0's r2: 0.62526
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[69]	training's l2: 24.8943	training's r2: 0.740272	valid_0's l2: 31.6884	valid_0's r2: 0.626842
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	training's l2: 24.5923	training's r2: 0.743423	valid_0's l2: 31.5672	valid_0's r2: 0.62827
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[71]	training's l2: 24.2856	training's r2: 0.746622	valid_0's l2: 31.3355	valid_0's r2: 0.630999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	training's l2: 23.9849	training's r2: 0.74976	valid_0's l2: 31.

[115]	training's l2: 15.6095	training's r2: 0.837142	valid_0's l2: 27.191	valid_0's r2: 0.679804
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[116]	training's l2: 15.4622	training's r2: 0.838679	valid_0's l2: 27.1759	valid_0's r2: 0.679981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[117]	training's l2: 15.3363	training's r2: 0.839993	valid_0's l2: 27.1709	valid_0's r2: 0.68004
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[118]	training's l2: 15.1876	training's r2: 0.841544	valid_0's l2: 27.118	valid_0's r2: 0.680664
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[119]	training's l2: 15.0481	training's r2: 0.843	valid_0's l2: 26.9973	valid_0's r2: 0.682084
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[120]	training's l2: 14.92	training's r2: 0.844337	valid_0's l2: 26.9892	valid_0's r2: 0.682179
[LightGBM] [Warning] No further splits with p

[167]	training's l2: 11.0118	training's r2: 0.885111	valid_0's l2: 25.6115	valid_0's r2: 0.698403
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	training's l2: 10.9525	training's r2: 0.88573	valid_0's l2: 25.5179	valid_0's r2: 0.699506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	training's l2: 10.9076	training's r2: 0.886198	valid_0's l2: 25.4606	valid_0's r2: 0.700181
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	training's l2: 10.8671	training's r2: 0.886621	valid_0's l2: 25.4052	valid_0's r2: 0.700833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[171]	training's l2: 10.8073	training's r2: 0.887245	valid_0's l2: 25.3851	valid_0's r2: 0.70107
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[172]	training's l2: 10.7449	training's r2: 0.887896	valid_0's l2: 25.3739	valid_0's r2: 0.701202
[LightGBM] [Warning] No further splits 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[227]	training's l2: 8.04423	training's r2: 0.916073	valid_0's l2: 24.4942	valid_0's r2: 0.71156
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[228]	training's l2: 8.02106	training's r2: 0.916314	valid_0's l2: 24.4506	valid_0's r2: 0.712074
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[229]	training's l2: 7.98257	training's r2: 0.916716	valid_0's l2: 24.4036	valid_0's r2: 0.712627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[230]	training's l2: 7.95263	training's r2: 0.917028	valid_0's l2: 24.3881	valid_0's r2: 0.71281
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[231]	training's l2: 7.92095	training's r2: 0.917359	valid_0's l2: 24.3755	valid_0's r2: 0.712958
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[232]	training's l2: 7.87928	training's r2: 0.917794	valid_0's

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[160]	training's l2: 10.0164	training's r2: 0.894295	valid_0's l2: 48.5528	valid_0's r2: 0.49607
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[161]	training's l2: 9.9582	training's r2: 0.894909	valid_0's l2: 48.4953	valid_0's r2: 0.496667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[162]	training's l2: 9.89391	training's r2: 0.895587	valid_0's l2: 48.4577	valid_0's r2: 0.497058
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[163]	training's l2: 9.82765	training's r2: 0.896287	valid_0's l2: 48.374	valid_0's r2: 0.497926
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[164]	training's l2: 9.75373	training's r2: 0.897067	valid_0's l2: 48.2355	valid_0's r2: 0.499364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[165]	training's l2: 9.68631	training's r2: 0.897778	valid_0's 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[245]	training's l2: 6.25144	training's r2: 0.934027	valid_0's l2: 46.1049	valid_0's r2: 0.521478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[246]	training's l2: 6.22266	training's r2: 0.934331	valid_0's l2: 46.1299	valid_0's r2: 0.521218
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[247]	training's l2: 6.20466	training's r2: 0.934521	valid_0's l2: 46.1277	valid_0's r2: 0.52124
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[248]	training's l2: 6.18213	training's r2: 0.934759	valid_0's l2: 46.1402	valid_0's r2: 0.521111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[249]	training's l2: 6.14767	training's r2: 0.935122	valid_0's l2: 46.1448	valid_0's r2: 0.521063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	training's l2: 6.12917	training's r2: 0.935318	valid_0'

[331]	training's l2: 4.23969	training's r2: 0.955258	valid_0's l2: 45.1103	valid_0's r2: 0.531801
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[332]	training's l2: 4.22172	training's r2: 0.955447	valid_0's l2: 45.0906	valid_0's r2: 0.532005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[333]	training's l2: 4.20046	training's r2: 0.955672	valid_0's l2: 45.0452	valid_0's r2: 0.532476
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[334]	training's l2: 4.1824	training's r2: 0.955862	valid_0's l2: 45.0437	valid_0's r2: 0.532491
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[335]	training's l2: 4.16192	training's r2: 0.956078	valid_0's l2: 45.0556	valid_0's r2: 0.532368
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[336]	training's l2: 4.14211	training's r2: 0.956287	valid_0's l2: 45.0358	valid_0's r2: 0.532573
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[416]	training's l2: 3.12932	training's r2: 0.966976	valid_0's l2: 44.2672	valid_0's r2: 0.540551
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[417]	training's l2: 3.11977	training's r2: 0.967076	valid_0's l2: 44.2746	valid_0's r2: 0.540474
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[418]	training's l2: 3.10384	training's r2: 0.967244	valid_0's l2: 44.2801	valid_0's r2: 0.540417
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[419]	training's l2: 3.09389	training's r2: 0.967349	valid_0's l2: 44.2735	valid_0's r2: 0.540486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[420]	training's l2: 3.08797	training's r2: 0.967412	valid_0's l2: 44.2768	valid_0's r2: 0.540451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[421]	training's l2: 3.07782	training's r2: 0.967519	valid_0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[504]	training's l2: 2.35254	training's r2: 0.975173	valid_0's l2: 44.2114	valid_0's r2: 0.54113
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[505]	training's l2: 2.34635	training's r2: 0.975238	valid_0's l2: 44.2079	valid_0's r2: 0.541167
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[506]	training's l2: 2.3401	training's r2: 0.975304	valid_0's l2: 44.2098	valid_0's r2: 0.541147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[507]	training's l2: 2.33221	training's r2: 0.975388	valid_0's l2: 44.1767	valid_0's r2: 0.541491
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[508]	training's l2: 2.32494	training's r2: 0.975464	valid_0's l2: 44.1632	valid_0's r2: 0.54163
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[509]	training's l2: 2.31859	training's r2: 0.975531	valid_0's 

In [10]:
df_res = pd.DataFrame(final_res)
df_res.to_csv("our_result.csv", header = ["id", "y"], index=False)